In [1]:
import numpy as np
from numpy.random import multivariate_normal
from numpy.linalg import norm
import pandas as pd
import seaborn as sb

In [2]:
P_a = [[-1, -1], [[2, 0.5], [0.5, 1]]]
P_b = [[1, -1] , [[1, -0.5], [-0.5, 2]]]
P_c = [[0, 1], [[1, 0], [0, 2]]]
params = [P_a, P_b, P_c]
num_dists = len(params)
means = [p[0] for p in params]
covs = [p[1] for p in params]
dist_means = np.array(means)
dist_covs = np.array(covs)

In [3]:
params_sigma = [0.5, 1, 2, 4, 8]
points_per_dist = 100
datasets = []
for sigma in params_sigma:
    dist_covs_mult = sigma*dist_covs
    obs = [multivariate_normal(dist_means[i], dist_covs_mult[i], size=points_per_dist) for i in range(num_dists)]
    dataset_sigma = np.concatenate(obs)
    datasets.append(dataset_sigma)

datasets = np.array(datasets)
# datasets

In [4]:
### K-means

DEBUG_KMEANS = False

def distances(points, means):
    distances = np.array([norm(points-m, axis=1) for m in means])
    return np.transpose(distances)

def objective(model, points):
    means = model.mu
    a = model.assignments(points)
    # takes a list of means, an assignment function f, and a collection of points, and returns the value of the objective function
    d = distances(points, means)
    distances_assignments = np.array([d[i][a[i]] for i in range(len(a))])
    if DEBUG_KMEANS:
        print(f'distances_assignments={distances_assignments}')
    return sum(distances_assignments**2)

class KMEANS:

    def __init__(self, seed=None):
        self.mu = None
        self.rng = np.random.default_rng(seed=seed)
    
    def assignments(self, points, d=None):
        if d is None:
            d = distances(points, self.mu)
        return np.argmin(d, axis=1)

    def train(self, k, points):
        self.mu = self.rng.random(size=(k, 2))

        if DEBUG_KMEANS:
            print(self.mu)

        obj = -1
        obj_new =  0
        while not obj == obj_new:
            obj = obj_new

            # 2. Calculate point assignments to clusters
            assignments = self.assignments(points)

            if DEBUG_KMEANS:
                print(f'points=\n{points}, \nassignments=\n{assignments}')

            # 3. Update means to be cluster centroids
            
            # create empty lists of points for each cluster, add points which have assignments to them 
            a_to_c = []
            for c in range(k):
                a_to_c.append([])
            for i in range(len(assignments)):
                a = assignments[i]
                a_to_c[a].append(points[i])

            # make python lists into np arrays for ease of mean computation
            for c in range(k):
                a_to_c[c] = np.array(a_to_c[c])

            # calculate new means for each cluster
            for c in range(k):
                if DEBUG_KMEANS:
                    print(f'For cluster {c}')
                    print(a_to_c[c])
                if len(a_to_c[c]) > 0: # only update mean if there was at least one point assigned to it
                    mean_new = np.mean(a_to_c[c], axis=0)
                    self.mu[c] = mean_new
                    if DEBUG_KMEANS:
                        print("cluster has at least one point assigned to it")
                        print(f'mean_new={mean_new}')
                
                if DEBUG_KMEANS:
                    print()

            # calculate new objective for the new means and their assigned points
            obj_new = objective(self, points)
            if DEBUG_KMEANS:
                print(f'means=\n{self.mu}, \nobj_new=\n{obj_new}\n')
        
        obj = obj_new 
        return obj

# points_test = datasets[0]
# model_kmeans = KMEANS(seed=42)
# model_kmeans.train(k=3, points=points_test)
# print('Objective=',objective(model_kmeans, points_test))
# import matplotlib.pyplot as plt
# a = model_kmeans.assignments(points=points_test)
# plt.scatter(points_test[:,0], points_test[:,1], c=a)

In [5]:
### GMM
from scipy.stats import multivariate_normal
from math import log
import random

DEBUG_GMM = False

class GMM:

    def __init__(self, seed=None):
        self.rng = np.random.default_rng(seed=seed)
        self.k = None
        self.phi = None
        self.mu = None
        self.Sigma = None
        self.w = None

    def prob_point(self, p, j):
        return multivariate_normal.pdf(p, self.mu[j], self.Sigma[j]) * self.phi[j]       

    def log_likelihood(self, points):
        # given the parameters for this model and points, calculates the log likelihood of those points
        result = 0
        for i in range(len(points)):
            inner_sum = 0 
            for j in range(num_dists):
                p_xi_given_zi = multivariate_normal.pdf(points[i], self.mu[j], self.Sigma[j])
                inner_sum += p_xi_given_zi * self.phi[j]
            result += log(inner_sum)
        return -result

    def assignments(self, points):
        # given the model parameters and some points, return the assignment for the points
        # points are assigned to the cluster which yields the highest probability of the point being there
        a = []
        for i in range(len(points)):
            inner_sum = 0 # TODO this does not currently do what the function says it does
            prob_max_idx = 0
            prob_max_val = self.prob_point(points[i], 0)
            for z in range(1,self.k):
                prob_i_z = self.prob_point(points[i], z)
                if prob_i_z > prob_max_val:
                    prob_max_idx = z
            a.append(prob_max_idx)
        return a 

    def train(self, k, points):
        # given a parameter k and points, trains this model to fit those points
        self.k = k
        self.phi = [1/num_dists for _ in range(num_dists)]
        # self.mu = self.rng.random(size=(k, 2))
        # self.mu = random.sample(points, k)
        self.mu = np.random.permutation(points)[:k]
        self.Sigma = np.array([np.identity(2) for _ in range(k)])
        # if DEBUG_GMM:
        #     print('Before GMM begins')
        #     print(f'phi=\n{self.phi},\nmu=\n{self.mu}, \nSigma=\n{self.Sigma}')
        obj = None
        obj_new = None
        while obj is None or obj_new / obj < 0.999:
            obj = obj_new

            # Expectation
            self.w = np.empty(shape=(len(points), num_dists))
            for i in range(len(points)):
                p = points[i]
                w_i_tot = 0
                for z in range(num_dists):
                    w_i_j = self.prob_point(p, z)
                    w_i_tot += w_i_j
                    self.w[i][z] = w_i_j
                for j in range(num_dists): # normalize each probability
                    self.w[i][j] /= w_i_tot

            # Maximization
            phi_new = np.empty(shape=(num_dists))
            mu_new = np.empty(shape=(num_dists, 2))
            Sigma_new = np.empty(shape=(num_dists, 2, 2))

            for j in range(self.k): # TODO speed up by using np functions or rethinking runtime
                w_all_j = self.w[:,j].reshape(len(points), 1)
                # w_all_j = self.w[:,j].reshape(len(points))
                sum_wj = sum(w_all_j[:, 0]) # sum over all i values of w_i_j
                phi_new[j] = sum_wj/len(points)
                mu_new[j] = sum(w_all_j * points)/sum_wj
                points_demeaned = points - self.mu[j]
                Sigma_new[j] = sum(w_all_j.reshape(len(points), 1, 1) * np.array([np.outer(points_demeaned[i], points_demeaned[i]) for i in range(len(points))]) )/sum_wj

            self.phi = phi_new
            self.mu = mu_new
            self.Sigma = Sigma_new

            # if DEBUG_GMM:
            #     print('After M step, we have:')
            #     print(f'phi=\n{self.phi},\nmu=\n{self.mu}, \nSigma=\n{self.Sigma}')

            # Calculate new objective (log likelihood)
            obj_new = self.log_likelihood(points)
            if DEBUG_GMM:
                print(f'obj_new={obj_new}')

        obj = obj_new
        return obj

points_test = datasets[0]

# model_gmm = GMM()
# model_gmm.train(k=3, points=datasets[1])
# model_gmm = GMM()
# model_gmm.train(k=3, points=points_test)
# a = model_gmm.assignments(points=points_test)
# plt.scatter(points_test[:,0], points_test[:,1], c=a)

In [6]:
def map_means_to_clusters(means, clusters):
    # takes sample means and maps them to their nearest true clusters
    # I choose to do this by assigning the mean and cluster which have the smallest distance to each other each time
    # then repeat this process for the remaining means and clusters
    map_r = {}
    taken_m = set()
    taken_c = set()
    for _ in range(len(means)):
        min_val = None 
        min_idx = None
        for i in range(len(means)):
            for j in range(len(clusters)):
                if i in taken_m or j in taken_c:
                    continue
                d =  norm(means[i]-clusters[j], axis=0)
                if min_val is None or d < min_val:
                    min_val = d
                    min_idx = (i,j)
        i, j = min_idx
        map_r[i] = j
        taken_m.add(i)
        taken_c.add(j)

    return [map_r[i] for i in range(len(means))]

In [7]:
# evaluate both methods on (i) the clustering objective (1) and (ii) the clustering accuracy. For each of the two criteria, plot the value achieved by each method against σ

method_names = ['k-means', 'GMM']
method_to_fn = { 'k-means':KMEANS, 'GMM': GMM }
models = { m:[] for m in method_names }
num_runs = 10
for m in method_names:
    models_m = []
    for i in range(len(params_sigma)):
        s = params_sigma[i]
        points_test = datasets[i]
        print(f'Running {m} for sigma={s}')
        
        runs = []
        for j in range(num_runs):
            model_m = method_to_fn[m](seed=42)
            obj = model_m.train(k=3, points=points_test)
            runs.append((model_m, obj))

        run_best = min(runs, key=lambda x: x[1])
        models[m].append(run_best)

Running k-means for sigma=0.5
Running k-means for sigma=1
Running k-means for sigma=2
Running k-means for sigma=4
Running k-means for sigma=8
Running GMM for sigma=0.5
Running GMM for sigma=1
Running GMM for sigma=2
Running GMM for sigma=4
Running GMM for sigma=8


In [8]:
### plot accuracy vs Sigma
# first, we define a mapping from the means to the original clusters
# then, for each point, we obtain assignments to means
# if mean->cluster = cluster from which point was drawn, then +1 correctly classified point

num_points = num_dists*points_per_dist
labels_true = []
for clust_label in range(num_dists):
    labels_true.extend([clust_label for _ in range(100)])
# print(labels_true)

tab = { m:{ 'Sigma':params_sigma, 'Objective Value':[], 'Accuracy':[] } for m in method_names }
for m in method_names:
    models_m = models[m]
    for i in range(len(params_sigma)):
        # print(f'For model {m}, sigma={params_sigma[i]},')
        model_m_sigma = models_m[i][0]
        # print(model_m_sigma)
        a = model_m_sigma.assignments(datasets[i])

        o = objective(model_m_sigma, datasets[i])
        tab[m]['Objective Value'].append(o)

        # print(a)
        mapping = map_means_to_clusters(model_m_sigma.mu, np.array(means))
        # print(mapping)
        a_mapped = [mapping[e] for e in a]
        # print(a_mapped)
        
        count_correct = 0
        for idx_a in range(len(a_mapped)):
            if labels_true[idx_a] == a_mapped[idx_a]:
                count_correct += 1
        accuracy = count_correct/num_points
        # print(f'Accuracy={accuracy}')
        tab[m]['Accuracy'].append(accuracy)

df_kmeans = pd.DataFrame(tab['k-means'])
df_gmm = pd.DataFrame(tab['GMM'])

In [9]:
df_kmeans

,Sigma,Objective Value,Accuracy
0,0.5,329.413801,0.846667
1,1.0,484.501959,0.720000
2,2.0,899.716767,0.610000
3,4.0,1888.004757,0.563333
4,8.0,3148.756013,0.550000


In [10]:
df_gmm

,Sigma,Objective Value,Accuracy
0,0.5,452.017711,0.743333
1,1.0,579.635779,0.716667
2,2.0,1289.609795,0.550000
3,4.0,3459.513697,0.473333
4,8.0,6319.726476,0.400000
